In [92]:
import cv2
import numpy as np
import random
from matplotlib import pyplot as plt
from pylab import mpl

# 指定默认字体,解决保存图像是负号'-'显示为方块的问题
mpl.rcParams['font.sans-serif'] = ['FangSong']
mpl.rcParams['axes.unicode_minus'] = False

In [93]:
def read_picture(car_name):
    path = 'E:\\homework\\DigitImage\\大作业12.8交\\车牌\\III类\\' + car_name
    image =cv2.imdecode(np.fromfile(path, dtype=np.uint8), -1)
    return image

In [94]:
def pre_deal(image):
    # 转化为灰度图像
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 高斯模糊，低通滤波
    gau_blur = cv2.GaussianBlur(gray_image, (3, 3), 0)
    # 中值滤波
#     median = cv2.medianBlur(gau_blur, 5)
    # sobel边缘检测
    sobel_img = cv2.Sobel(gau_blur, cv2.CV_16S, 1, 0)
    sobel_img = cv2.convertScaleAbs(sobel_img)
    
    mean = (np.max(image) + np.min(image)) / 2
    
    # ostu自适应阈值处理二值化 cv2.THRESH_OTSU|
    _, otsu_car = cv2.threshold(sobel_img, mean, 255, cv2.THRESH_BINARY)

    # 闭操作（5,5）
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 10))
#     close_car = cv2.morphologyEx(otsu_car,cv2.MORPH_CLOSE,kernel)
    
    
    # 核函数
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 7))
    # 膨胀
    dilation = cv2.dilate(otsu_car, element2, iterations=1)
    # 腐蚀
    erosion = cv2.erode(dilation, element1, iterations=1)
    # 膨胀
#     dilation2 = cv2.dilate(erosion, element2, iterations=1)
    cv2.imshow("binary2", erosion)
    cv2.waitKey(0)
    return erosion

In [95]:
def contours_deal(car_image, image):
    
    save = []  #存储合理轮廓
    rectall = []  #存储对应的在最小面积矩形
    
    # 边缘提取
    contours, hierarchy = cv2.findContours(car_image, cv2.RETR_EXTERNAL,
                                           cv2.CHAIN_APPROX_NONE)
    for contour in contours:
        rectangle = cv2.minAreaRect(contour)
        if size_select(rectangle):
            save.append(contour)
            rectall.append(rectangle)
    cv2.drawContours(image, save, -1, (0, 255, 0), 3)
    cv2.imshow("binary2", image)
    cv2.waitKey(0)
    return save, rectall

def floodfill(car_image, image):
    save, rectall = contours_deal(car_image, image)
    for step, rect in enumerate(rectall):
        x, y = rect[0]   #x:列数，y:行数
        x = int(x)
        y = int(y)
 
        cv2.circle(image, (x, y), 3, (0, 255, 0), 2)

        width, height = rect[1]
        minimum = width if width < height else height
        minimum = 0.5 * minimum

        h,w = image.shape[:2]  #600,


        mask = np.zeros((h + 2, w + 2), dtype=np.uint8)

        for i in range(10):
            seed_x = int(x+0.5*(np.random.randint(0,100000-1)%int(minimum)-(minimum/2)))
            seed_y = int(y+0.5*(np.random.randint(0,100000-1)%int(minimum)-(minimum/2)))

            cv2.circle(image,(seed_x,seed_y),1,(0,255,255))

            loDiff = 7.95
            upDiff = 30
            Connectivity = 4
            flag = Connectivity + (255<<8) + cv2.FLOODFILL_MASK_ONLY
            cv2.floodFill(image,mask,(seed_x,seed_y),(255,0,0),(loDiff,loDiff,loDiff),(upDiff,upDiff,upDiff),flag)

        # cv2.imshow(str(step),mask)

        points = []
        row,column = mask.shape

        for i in range(row):
            for j in range(column):
                if mask[i][j]==255:
                    points.append((j,i))   #点应该输入点坐标（列，行）
        points = np.asarray(points)
        new_rect = cv2.minAreaRect(points)
        print(new_rect)
        if size_select(new_rect):
            print('hhhhhhhhhhhhhhh')
            # 宽,高
            x,  y = new_rect[0]
            new_width, new_height = new_rect[1]
            angel = new_rect[2]
            point1 = cv2.boxPoints(new_rect)[0]
            point2 = cv2.boxPoints(new_rect)[1]
            point3 = cv2.boxPoints(new_rect)[2]
            point4 = cv2.boxPoints(new_rect)[3]

            rotate = cv2.getRotationMatrix2D((x,y),90+angel,1)
            res = cv2.warpAffine(image,rotate,image.shape[:2])

            #img,(列，行)，（中心）
            res = cv2.getRectSubPix(res,(int(new_height),int(new_width)),(x,y))

            #img,(列，行)
            res = cv2.resize(res,(105,25),interpolation=cv2.INTER_AREA)
            res = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
            res = cv2.GaussianBlur(res,(3,3),0)
            res = cv2.equalizeHist(res)
            print(res)
            cv2.imshow("dd", res)
            cv2.waitKey(0)

In [96]:
def size_select(min_rec):
    error = 0.4
    ratio = 3.143
    min_area = 25 * 25 * ratio
    max_area = 100 * 100 * ratio
    min_ratio = ratio * (1 - error)
    max_ratio = ratio * (1 + error)
    w, h = min_rec[1]
    if h == 0 or w == 0:
        return False
    area = h * w
    r = w / h
    if r < 1:
        r = h / w
    
    if (area < min_area or area > max_area) or (r < min_ratio or r > max_ratio):
        return False
    else:
        return True
    

In [97]:
def show_pic(image):
    figure = plt.figure(figsize=(10, 6))
    
    cv2.imshow("binary2", image)
    cv2.waitKey(0)
#     plt.imshow(close_car, cmap='gray')
#     plt.xticks([])
#     plt.yticks([])
#     plt.show()

In [98]:
def main():
    name = ['贵OA2803.jpg', '晋A00002.jpg', '晋AAS999.jpg', '晋C88888.jpg', '京K89779.jpg',
           '粤A3Y347.jpg', '云AU7526.jpg']
    for i in name:
        image = read_picture(i)
        deal_car = pre_deal(image)
        floodfill(deal_car, image)
#         show_pic(deal_car)
main()

((330.5, 239.5), (7.0, 7.0), -0.0)
((604.82763671875, 499.9309997558594), (9.191452026367188, 8.140999794006348), -23.198589324951172)
((280.0, 255.0), (10.0, 14.0), -0.0)
((171.0, 251.5), (6.0, 7.0), -0.0)
((85.99999237060547, 12.0), (7.155416965484619, 8.049844741821289), -63.43495178222656)
((257.8500061035156, 306.0500183105469), (6.640782833099365, 10.75174331665039), -18.434947967529297)
((448.5, 232.5), (5.0, 5.0), -90.0)
((209.2783660888672, 245.6237335205078), (15.12865924835205, 18.27623176574707), -66.03751373291016)
((163.8119659423828, 168.67922973632812), (15.498927116394043, 13.70291519165039), -3.814074993133545)
((384.5000305175781, 240.49996948242188), (6.833310127258301, 10.738059043884277), -12.528807640075684)
((516.0, 93.5), (4.0, 3.0), -0.0)
((18.0, 65.0), (8.0, 8.0), -0.0)
((542.7567138671875, 300.5405578613281), (7.562353610992432, 5.918363571166992), -9.462321281433105)
((603.3649291992188, 290.8108215332031), (4.767570495605469, 6.904757499694824), -80.537673